In [31]:
import pandas as pd
import numpy as np

data = pd.read_csv("IMDB-Movie-Data.csv")
df = pd.DataFrame(data)

### 1. Top‐3 movies with the highest ratings in 2016?

In [32]:
#find the top 3 ratings
movies_2016 = df[df['Year'] == 2016]
top3_rating_2016 = movies_2016['Rating'].drop_duplicates().nlargest(3)

top_movies = movies_2016[movies_2016['Rating'].isin(top3_rating_2016)]

top_movies.index = range(1, len(top_movies) + 1)
top_movies[['Title', 'Rating']]

,Title,Rating
1,Dangal,8.8
2,Kimi no na wa,8.6
3,Koe no katachi,8.4


### 2. The actor generating the highest average revenue?

In [33]:
df_exploded = df.assign(Actors=df['Actors'].str.split('|')).explode('Actors')


df_exploded['Actors'] = df_exploded['Actors'].str.strip()

#group by actor and calculate the average revenue for each actor
actor_avg_revenue = df_exploded.groupby('Actors')['Revenue (Millions)'].mean()

#find the actor(s) with the highest average revenue
max_avg_revenue = actor_avg_revenue.max()
top_actors = actor_avg_revenue[actor_avg_revenue == max_avg_revenue]
top_actors_df = top_actors.reset_index().rename(columns={'index': 'Actor', 0: 'average revenue'})
top_actors_df.index = range(1, len(top_actors_df) + 1)
top_actors_df

,Actors,Revenue (Millions)
1,Daisy Ridley,936.63
2,John Boyega,936.63


### 3.  The average rating of Emma Watson’s movies?

In [34]:
# Filter for movies with Emma Watson in the 'Actors' column
df_emma = df[df['Actors'].str.contains('Emma Watson', case=False, na=False)]

average_rating = df_emma['Rating'].mean()
round(average_rating,5)

7.175

### 4. Top‐3 directors who collaborate with the most actors?

In [35]:
df_exploded = df.assign(Actors=df['Actors'].str.split('|')).explode('Actors')

df_exploded['Actors'] = df_exploded['Actors'].str.strip()

#group by director and count the number of unique actors
director_actor_counts = df_exploded.groupby('Director')['Actors'].nunique()


top_counts = director_actor_counts.drop_duplicates().nlargest(3)

#select all directors with actor counts matching the top 3 counts
top_directors = director_actor_counts[
    director_actor_counts.isin(top_counts)
].sort_values(ascending=False)

top_directors_df = top_directors.reset_index().rename(columns={'index': 'Actor', 0: 'numbers of collaboration'})
top_directors_df.index = range(1, len(top_directors_df) + 1)
top_directors_df

,Director,Actors
1,Ridley Scott,28
2,M. Night Shyamalan,24
3,Danny Boyle,20
4,Paul W.S. Anderson,20


### 5. Top‐2 actors playing in the most genres of movies?

In [36]:
df_exploded = df.assign(
    Actors=df['Actors'].str.split('|'),
    Genre=df['Genre'].str.split('|')
).explode('Actors').explode('Genre')


df_exploded['Actors'] = df_exploded['Actors'].str.strip()
df_exploded['Genre'] = df_exploded['Genre'].str.strip()

#group by actors and count the number of unique genres
actor_genre_counts = df_exploded.groupby('Actors')['Genre'].nunique()

#sort by genre count in descending order
actor_genre_counts = actor_genre_counts.sort_values(ascending=False)

#identify the top 2 highest genre counts
top_2_counts = actor_genre_counts.drop_duplicates().nlargest(2)

#select all actors with these top 2 counts
top_actors = actor_genre_counts[actor_genre_counts.isin(top_2_counts)]


top_actors_df = top_actors.reset_index().rename(columns={'index': 'Actor', 0: 'Unique Genre Count'})
top_actors_df.index = range(1, len(top_actors_df) + 1)
top_actors_df

,Actors,Genre
1,Brad Pitt,14
2,Hugh Jackman,13
3,Scarlett Johansson,13
4,Chloe Grace Moretz,13
5,Amy Adams,13
6,Johnny Depp,13


### 6. Top‐3 actors whose movies lead to the largest maximum gap of years?

In [37]:
df_exploded = df.assign(Actors=df['Actors'].str.split('|')).explode('Actors')

#convert 'Year' to numeric (in case it's not)
df_exploded['Year'] = pd.to_numeric(df_exploded['Year'], errors='coerce')

#group by actor and calculate the min and max year
actor_year_gap = df_exploded.groupby('Actors')['Year'].agg(['min', 'max'])
actor_year_gap['max_gap'] = actor_year_gap['max'] - actor_year_gap['min']

#identify the top 3 maximum gaps
top_gaps = actor_year_gap['max_gap'].drop_duplicates().nlargest(3)

#select all actors with these top gaps
top_3_actors = actor_year_gap[actor_year_gap['max_gap'].isin(top_gaps)]

#sort the result by max_gap in descending order
top_3_actors = top_3_actors.sort_values(by='max_gap', ascending=False)

top_3_actors_list = top_3_actors[['max_gap']].reset_index().rename(columns={'Actors': 'Actor', 'max_gap': 'Max Gap'})
top_3_actors_list.index = range(1, len(top_3_actors_list) + 1)
top_3_actors_list

,Actor,Max Gap
1,Michelle Monaghan,10
2,Ellen Burstyn,10
3,Hugh Jackman,10
4,Jack Davenport,10
5,Jennifer Connelly,10
...,...,...
143,Paul Giamatti,8
144,Meryl Streep,8
145,Channing Tatum,8
146,Mark Strong,8


### 7. Find all actors who collaborate with Johnny Depp in direct and indirect ways
Example:
A collaborates with B
B collaborates with C and 
D C collaborates with E and 
F D collaborates with A and 
G G collaborates with H

All actors directly and indirectly collaborating with A include: [B, C, D, E, F, G, H]

In [38]:
df_exploded = df.assign(Actors=df['Actors'].str.split('|')).explode('Actors')
df_exploded['Actors'] = df_exploded['Actors'].str.strip()

#build a graph via a dictionary of sets
graph = {}
for _, group in df_exploded.groupby('Title'):
    actors = group['Actors'].tolist()
    for i in range(len(actors)):
        for j in range(i + 1, len(actors)):
            #add undirected edges between j, i
            graph.setdefault(actors[i], set()).add(actors[j])
            graph.setdefault(actors[j], set()).add(actors[i])

#dfs all the set and update the collaborative set for jonny deep
def dfs(actor, visited):
    if actor in visited:
        return
    visited.add(actor)
    for neighbor in graph.get(actor, []):
        dfs(neighbor, visited)

visited = set()
if 'Johnny Depp' in graph:
    dfs('Johnny Depp', visited)
    
visited.remove('Johnny Depp')#do not collaborate with himself
collaborators_df = pd.DataFrame(visited, columns=['Collaborator'])
collaborators_df.index = range(1, len(collaborators_df) + 1)
collaborators_df

,Collaborator
1,Quinton Aaron
2,Michael Kelly
3,Kevin Kline
4,Sean Faris
5,Susan Sarandon
...,...
1570,Deborah Kara Unger
1571,Ana de Armas
1572,Alexander Black
1573,T.I.
